## Data Export

In [248]:
import json
import pandas as pd
import numpy as np
import json
import warnings
import geojson
from prophet import Prophet
import xlsxwriter

import pandas as pd
import numpy as np
import jinja2
import pdfkit
from xhtml2pdf import pisa
import plotly.express as px
import plotly.graph_objects as go
#for worldmap
from urllib.request import urlopen
import json
import kaleido

### Sources

#### Our World in Data

In [249]:
##### Data #####
df = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')

path_to_file = 'countries.geojson'
with open(path_to_file) as f:
    world_geojson = geojson.load(f)

# Sort dataframe by date
df = df.sort_values(by="date")

# Convert date to pd datetime object
df['date'] = pd.to_datetime(df.date,format='%Y-%m-%d')

# Extracting day month year from date
#df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['month_year'] = df['date'].dt.to_period('M')
df = df[df.date > "2020-01-22"]

#df['inzidenz'] = df.new_cases_per_million.rolling(7).mean()
df = df[['iso_code','date','month_year', 'month', 'year', 'location', 'continent','new_cases_per_million', 'total_cases','new_deaths_per_million','total_deaths']]

df = df.groupby(['iso_code','month_year','month','year','location','continent']).sum().reset_index()
df = df.sort_values(by="month_year")
df = df[['year','month_year','location','iso_code','total_cases','total_deaths']]
df.columns = ['Jahr','Jahr-Monat','Land','Iso_Code','Infizierte','Tote']

# Create own dataframe for world and germany
germany = df[df.Land == 'Germany']
germany = pd.DataFrame(germany.iloc[-1]).transpose()

#### DIVI

In [250]:
##### DIVI #####

divi = pd.read_csv("https://diviexchange.blob.core.windows.net/%24web/zeitreihe-tagesdaten.csv")

path_to_file = 'landkreise.geojson'
with open(path_to_file) as f:
    land_geojson= geojson.load(f)

codes = divi['gemeindeschluessel'].astype('int').drop_duplicates()
codes = codes.tolist()

dict_codes = {}

for x in range(0,len(codes)):
   
    code = codes[x]

    if code < 10000:
        dict_codes[code] = "0"+str(code)
    else:
        dict_codes[code] = str(code)
    

divi["id"] = divi["gemeindeschluessel"].apply(lambda x: dict_codes[x])
bundesland = ["Schleswig-Holstein","Hamburg","Niedersachsen","Bremen","Nordrhein-Westfalen","Hessen","Rheinland-Pfalz","Baden-Württemberg","Bayern","Saarland","Berlin","Brandenburg","Mecklenburg-Vorpommern","Sachsen","Sachsen-Anhalt","Thüringen"]
code = range(1,17)
bund_code = dict(zip(code, bundesland))

divi["bundesland_name"] = divi["bundesland"].apply(lambda x: bund_code[x])

dict_landkreise = {}
for feature in land_geojson['features']:
    dict_landkreise[feature["properties"]["AGS"]] = feature["properties"]["GEN"]

divi["name"] = divi["id"].apply(lambda x: dict_landkreise[x])

divi = divi[['date','id','bundesland_name','name','faelle_covid_aktuell','faelle_covid_aktuell_invasiv_beatmet', 'betten_frei', 'betten_belegt']]
divi.columns = ['Datum','Id','Bundesland','Landkreis','Infizierte','Infizierte invasiv beatmet','Freie Betten','Belegte Betten']

#### RKI

In [251]:
##### RKI #####

rki = pd.read_csv("https://raw.githubusercontent.com/robert-koch-institut/COVID-19-Impfungen_in_Deutschland/master/Aktuell_Deutschland_Landkreise_COVID-19-Impfungen.csv")
rki = rki[rki.LandkreisId_Impfort != "u"]
rki.LandkreisId_Impfort = rki.LandkreisId_Impfort.astype('int')

lk = rki.LandkreisId_Impfort.drop_duplicates().tolist()

# Create dictionary to add id 
dict_ids = {}
for x in lk:
    if x < 10000:
        dict_ids[x] = "0"+str(x)
    else:
        dict_ids[x] = str(x)

rki['Id'] = rki["LandkreisId_Impfort"].apply(lambda x: dict_ids[x])
rki = rki.drop('LandkreisId_Impfort',axis=1)
rki = rki[rki.Id != '17000']

rki["Name"] = rki["Id"].apply(lambda x: dict_landkreise[x])
rki = pd.merge(rki,divi[['Id','Bundesland']].drop_duplicates(), left_on=["Id"], right_on=["Id"], how="left")
rki.loc[rki.Impfschutz == 1, "Impfschutz"] = "Erstimpfung"
rki.loc[rki.Impfschutz == 2, "Impfschutz"] = "Zweitimpfung"
rki.loc[rki.Impfschutz == 3, "Impfschutz"] = "Boosterimpfung"

rki.columns = ['Datum','Altergruppe','Impfschutz','Anzahl','Id','Landkreis','Bundesland']

/Users/hannaziegler/opt/anaconda3/envs/jbook/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning:

Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.



### Abbildungen

#### Fall-und Todeszahlen 2020

In [252]:
df[df.Jahr == 2020]

,Jahr,Jahr-Monat,Land,Iso_Code,Infizierte,Tote
1685,2020,2020-01,United Kingdom,GBR,2.0,0.0
2187,2020,2020-01,India,IND,2.0,0.0
2901,2020,2020-01,Macao,MAC,45.0,0.0
1597,2020,2020-01,France,FRA,30.0,0.0
1227,2020,2020-01,Germany,DEU,18.0,0.0
...,...,...,...,...,...,...
1977,2020,2020-12,Guatemala,GTM,4047232.0,139914.0
2691,2020,2020-12,Liberia,LBR,53646.0,2573.0
1311,2020,2020-12,Denmark,DNK,3750144.0,31323.0
4965,2020,2020-12,Vatican,VAT,837.0,0.0


#### Infizierte und Tote 2021

In [253]:
df[df.Jahr == 2021]

,Jahr,Jahr-Monat,Land,Iso_Code,Infizierte,Tote
4369,2021,2021-01,Suriname,SUR,230537.0,4340.0
866,2021,2021-01,Chile,CHL,20661911.0,540938.0
2568,2021,2021-01,Saint Kitts and Nevis,KNA,1072.0,0.0
2594,2021,2021-01,South Korea,KOR,2220738.0,37988.0
83,2021,2021-01,Anguilla,AIA,467.0,0.0
...,...,...,...,...,...,...
4213,2021,2021-12,El Salvador,SLV,3749302.0,117859.0
1080,2021,2021-12,Cape Verde,CPV,1203771.0,10891.0
1128,2021,2021-12,Cuba,CUB,29883178.0,257734.0
608,2021,2021-12,Belize,BLZ,968822.0,18239.0


#### Infizierte und Tote 2022

In [254]:
df[df.Jahr == 2021]

,Jahr,Jahr-Monat,Land,Iso_Code,Infizierte,Tote
4369,2021,2021-01,Suriname,SUR,230537.0,4340.0
866,2021,2021-01,Chile,CHL,20661911.0,540938.0
2568,2021,2021-01,Saint Kitts and Nevis,KNA,1072.0,0.0
2594,2021,2021-01,South Korea,KOR,2220738.0,37988.0
83,2021,2021-01,Anguilla,AIA,467.0,0.0
...,...,...,...,...,...,...
4213,2021,2021-12,El Salvador,SLV,3749302.0,117859.0
1080,2021,2021-12,Cape Verde,CPV,1203771.0,10891.0
1128,2021,2021-12,Cuba,CUB,29883178.0,257734.0
608,2021,2021-12,Belize,BLZ,968822.0,18239.0


#### Infizierte und Tote Deutschland

In [255]:
germany

,Jahr,Jahr-Monat,Land,Iso_Code,Infizierte,Tote
1252,2022,2022-02,Germany,DEU,111265207.0,1188349.0


##### Bettenbelegung

In [256]:
bettenbelegung = divi[(divi.Datum == max(divi.Datum))].groupby('Datum').sum().reset_index()
bettenbelegung = bettenbelegung[['Datum','Freie Betten','Belegte Betten']]
bettenbelegung

,Datum,Freie Betten,Belegte Betten
0,2022-02-11,3884,21036


#### Landkreise ohne freie Betten

In [257]:
lk_ohne_bett =  divi[(divi.Datum == max(divi.Datum))&(divi['Freie Betten'] == 0)&(divi['Infizierte'] > 0)]
lk_ohne_bett = lk_ohne_bett[['Datum','Bundesland','Landkreis','Infizierte','Freie Betten']]
lk_ohne_bett

,Datum,Bundesland,Landkreis,Infizierte,Freie Betten
260404,2022-02-11,Schleswig-Holstein,Schleswig-Flensburg,1,0
260425,2022-02-11,Niedersachsen,Schaumburg,3,0
260486,2022-02-11,Nordrhein-Westfalen,Coesfeld,4,0
260514,2022-02-11,Hessen,Darmstadt-Dieburg,13,0
260515,2022-02-11,Hessen,Groß-Gerau,8,0
260518,2022-02-11,Hessen,Main-Taunus-Kreis,1,0
260519,2022-02-11,Hessen,Odenwaldkreis,11,0
260547,2022-02-11,Rheinland-Pfalz,Bernkastel-Wittlich,2,0
260563,2022-02-11,Rheinland-Pfalz,Donnersbergkreis,1,0
260578,2022-02-11,Baden-Württemberg,Hohenlohekreis,1,0


#### Impfstatus 

In [258]:
impfstatus = rki.groupby(['Bundesland','Impfschutz']).sum('Anzahl').reset_index()
impfstatus

,Bundesland,Impfschutz,Anzahl
0,Baden-Württemberg,Boosterimpfung,6057452
1,Baden-Württemberg,Erstimpfung,7680317
2,Baden-Württemberg,Zweitimpfung,8113274
3,Bayern,Boosterimpfung,6873559
4,Bayern,Erstimpfung,9038699
5,Bayern,Zweitimpfung,9528928
6,Berlin,Boosterimpfung,2064403
7,Berlin,Erstimpfung,2696447
8,Berlin,Zweitimpfung,2780769
9,Brandenburg,Boosterimpfung,1222415


#### Impfquoten

In [259]:
#Impfquoten
quoten = pd.read_csv('https://raw.githubusercontent.com/robert-koch-institut/COVID-19-Impfungen_in_Deutschland/master/Aktuell_Deutschland_Impfquoten_COVID-19.csv')
quoten_land = quoten[(quoten.Bundesland!="Deutschland")&(quoten.Bundesland!="Bundesressorts")]
quoten_land = quoten_land[['Datum', 'Bundesland','Impfquote_gesamt_min1','Impfquote_gesamt_boost']]
quoten_land.columns = ['Datum','Bundesland','Impfquote Erstimpfung','Impfquote Boosterimpfung']
quoten_land

,Datum,Bundesland,Impfquote Erstimpfung,Impfquote Boosterimpfung
1,2022-02-10,Schleswig-Holstein,80.0,63.6
2,2022-02-10,Hamburg,81.5,53.9
3,2022-02-10,Niedersachsen,77.9,59.6
4,2022-02-10,Bremen,89.7,61.6
5,2022-02-10,Nordrhein-Westfalen,79.9,57.7
6,2022-02-10,Hessen,76.3,52.9
7,2022-02-10,Rheinland-Pfalz,77.0,55.5
8,2022-02-10,Baden-Württemberg,73.6,54.5
9,2022-02-10,Bayern,73.6,53.2
10,2022-02-10,Saarland,82.1,62.8


#### Forecast Fallzahlen

In [ ]:
cases_prophet= divi[['Datum','Infizierte']].groupby('Datum').sum().reset_index()
cases_prophet.columns = ['ds','y']

m = Prophet(interval_width=0.95,changepoint_prior_scale=0.01,seasonality_mode='multiplicative',daily_seasonality=True,weekly_seasonality=True,yearly_seasonality=True,changepoint_range=0.9)
m.fit(cases_prophet)
future = m.make_future_dataframe(periods=90)
forecast = m.predict(future)
forecast['y'] = cases_prophet['y']

pred = forecast[forecast.y.isna()]
pred.loc[pred['yhat'] < 0, 'yhat'] = 0
pred.loc[pred['yhat_lower'] < 0, 'yhat_lower'] = 0
pred.loc[pred['yhat_upper'] < 0, 'yhat_upper'] = 0

pred = pred[['ds','yhat']]
pred.columns = ['Datum','Infizierte']
pred

#### Excel Writer

In [260]:
writer = pd.ExcelWriter('covid19.xlsx',engine='xlsxwriter')

df[df.Jahr == 2020].to_excel(writer,sheet_name='Entwicklung Global 2020',startrow=1,index=False)
df[df.Jahr == 2021].to_excel(writer,sheet_name='Entwicklung Global 2021',startrow=1,index=False)
df[df.Jahr == 2022].to_excel(writer,sheet_name='Entwicklung Global 2022',startrow=1,index=False)
germany.to_excel(writer,sheet_name='Entwicklung Deutschland',startrow=1,index=False)
bettenbelegung.to_excel(writer,sheet_name='Betten frei_belegt',startrow=1,index=False)
lk_ohne_bett.to_excel(writer,sheet_name='LK ohne Betten',startrow=1,index=False)
impfstatus.to_excel(writer,sheet_name='Impfstatus',startrow=1,index=False)
quoten_land.to_excel(writer,sheet_name='Impfquoten',startrow=1,index=False)
pred.to_excel(writer,sheet_name='Prognose Infizierte',startrow=1,index=False)

In [261]:
book = writer.book
bold = book.add_format({'bold':True,'size':16})
sheets = {'Entwicklung Global 2020':'Globale Entwicklung der COVID-19 Pandemie in 2020','Entwicklung Global 2021':'Globale Entwicklung der COVID-19 Pandemie in 2021','Entwicklung Global 2022':'Globale Entwicklung der COVID-19 Pandemie in 2022','Entwicklung Deutschland':'Gesamtentwicklung der COVID-19 Pandemie in Deutschland','Betten frei_belegt':'Bettenbelegung','LK ohne Betten':'Anzahl Infizierter in Landkreisen ohne freie Intensivbetten','Impfstatus':'Impfstatus je Bundesland','Impfquoten':'Impfquoten je Bundesland','Prognose Infizierte':'Wie entwickeln sich die Fallzahlen, wenn die Zahl der Impfungen stagniert?'}

for key in sheets.keys():
    sheet = writer.sheets[key]
    sheet.write('A1',sheets[key],bold)

writer.save()

### HTML

In [184]:

##### Data #####
df = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')

path_to_file = 'countries.geojson'
with open(path_to_file) as f:
    world_geojson = geojson.load(f)

# Sort dataframe by date
df = df.sort_values(by="date")

# Convert date to pd datetime object
df['date'] = pd.to_datetime(df.date,format='%Y-%m-%d')

# Extracting day month year from date
#df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['month_year'] = df['date'].dt.to_period('M')
df = df[df.date > "2020-01-22"]

#df['inzidenz'] = df.new_cases_per_million.rolling(7).mean()
df = df[['iso_code','date','month_year', 'month', 'year', 'location', 'continent','new_cases_per_million', 'total_cases','new_deaths_per_million','total_deaths']]

df = df.groupby(['iso_code','month_year','month','year','location','continent']).sum().reset_index()
df = df.sort_values(by="month_year")

numdate2020= [x for x in range(len(df[df.year == 2020].month_year.unique()))]
numdate2021= [x for x in range(len(df[df.year == 2021].month_year.unique()))]
numdate2022= [x for x in range(len(df[df.year == 2022].month_year.unique()))]

# Create own dataframe for world and germany

germany = df[df.location == 'Germany']
################################################################

##### DIVI #####

divi = pd.read_csv("https://diviexchange.blob.core.windows.net/%24web/zeitreihe-tagesdaten.csv")

path_to_file = 'landkreise.geojson'
with open(path_to_file) as f:
    land_geojson= geojson.load(f)

codes = divi['gemeindeschluessel'].astype('int').drop_duplicates()
codes = codes.tolist()

dict_codes = {}

for x in range(0,len(codes)):
   
    code = codes[x]

    if code < 10000:
        dict_codes[code] = "0"+str(code)
    else:
        dict_codes[code] = str(code)
    

divi["id"] = divi["gemeindeschluessel"].apply(lambda x: dict_codes[x])
bundesland = ["Schleswig-Holstein","Hamburg","Niedersachsen","Bremen","Nordrhein-Westfalen","Hessen","Rheinland-Pfalz","Baden-Württemberg","Bayern","Saarland","Berlin","Brandenburg","Mecklenburg-Vorpommern","Sachsen","Sachsen-Anhalt","Thüringen"]
code = range(1,17)
bund_code = dict(zip(code, bundesland))

divi["bundesland_name"] = divi["bundesland"].apply(lambda x: bund_code[x])

dict_landkreise = {}
for feature in land_geojson['features']:
    dict_landkreise[feature["properties"]["AGS"]] = feature["properties"]["GEN"]

divi["name"] = divi["id"].apply(lambda x: dict_landkreise[x])

divi = divi[['date','id','bundesland_name','name','faelle_covid_aktuell','faelle_covid_aktuell_invasiv_beatmet', 'betten_frei', 'betten_belegt']]

################################################################

##### RKI #####

rki = pd.read_csv("https://raw.githubusercontent.com/robert-koch-institut/COVID-19-Impfungen_in_Deutschland/master/Aktuell_Deutschland_Landkreise_COVID-19-Impfungen.csv")
rki = rki[rki.LandkreisId_Impfort != "u"]
rki.LandkreisId_Impfort = rki.LandkreisId_Impfort.astype('int')

lk = rki.LandkreisId_Impfort.drop_duplicates().tolist()

# Create dictionary to add id 
dict_ids = {}
for x in lk:
    if x < 10000:
        dict_ids[x] = "0"+str(x)
    else:
        dict_ids[x] = str(x)

rki['id'] = rki["LandkreisId_Impfort"].apply(lambda x: dict_ids[x])
rki = rki.drop('LandkreisId_Impfort',axis=1)
rki = rki[rki.id != '17000']

rki["Name"] = rki["id"].apply(lambda x: dict_landkreise[x])
rki = pd.merge(rki,divi[['id','bundesland_name']].drop_duplicates(), left_on=["id"], right_on=["id"], how="left")
rki.loc[rki.Impfschutz == 1, "Impfschutz"] = "Erstimpfung"
rki.loc[rki.Impfschutz == 2, "Impfschutz"] = "Zweitimpfung"
rki.loc[rki.Impfschutz == 3, "Impfschutz"] = "Boosterimpfung"
################################################################


##### Quoten aktuell Bundesland #####

quoten = pd.read_csv('https://raw.githubusercontent.com/robert-koch-institut/COVID-19-Impfungen_in_Deutschland/master/Aktuell_Deutschland_Impfquoten_COVID-19.csv')
quoten_land = quoten[(quoten.Bundesland!="Deutschland")&(quoten.Bundesland!="Bundesressorts")]

/Users/hannaziegler/opt/anaconda3/envs/jbook/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning:

Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.



#### Abbildung 1.1: Monatlich Infizierte pro Land im zeitlichen Verlauf(2020,2021,2022)

In [186]:
#text
fig1_1_2020_cases_title='Wie haben sich die Fallzahlen im Laufe der Zeit entwickelt??'
fig1_1_2020_cases_description='''Die Grafik zeigt, welche Länder die höchsten Corona-Fallzahlen in 2020 verzeichneten. Die meisten Fälle in Summe verzeichnete die <b>USA (1,7 Billionen)</b>, gefolgt von <b>Indien (1,0 Billionen)</b> und <b> Brasilien (892 Millionen) </b>.'''

In [187]:
#grafik
filtered_df = df[df.year == 2020].groupby(['iso_code','year','location','continent']).sum().reset_index()

fig1_1_2020_cases = px.choropleth(filtered_df, 
            locations = 'iso_code',
            color = 'total_cases', 
            locationmode='ISO-3',
            scope='world',
            height=600,
            color_continuous_scale='purd',
            labels={'iso_code':'Land'
                }
            )
fig1_1_2020_cases.update_traces(hovertext='location')

fig1_1_2020_cases.update_layout(coloraxis_showscale=False,
    title = {'text':"Corona-Infizierte in Summe in 2020", 
        'x':0.5,
        'y':0.95,
        'xanchor': 'center',
        'yanchor': 'top'
        },
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig1_1_2020_cases.update_geos(fitbounds="locations", visible=False)

In [188]:
filtered_df = df[df.year == 2021].groupby(['iso_code','year','location','continent']).sum().reset_index()

fig1_1_2021_cases = px.choropleth(filtered_df, 
            locations = 'iso_code',
            color = 'total_cases', 
            locationmode='ISO-3',
            scope='world',
            height=600,
            color_continuous_scale='purd',
            labels={'iso_code':'Land'
                }
            )
fig1_1_2021_cases.update_traces(hovertext='location')

fig1_1_2021_cases.update_layout(coloraxis_showscale=False,
    title = {'text':"Corona-Infizierte in Summe in 2021", 
        'x':0.5,
        'y':0.95,
        'xanchor': 'center',
        'yanchor': 'top'
        },
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig1_1_2021_cases.update_geos(fitbounds="locations", visible=False)


In [189]:
#text
fig1_1_2022_cases_title='Wie haben sich die Fallzahlen bis heute entwickelt??'
fig1_1_2022_cases_description='''Die Grafik zeigt, dass die Top 3 Länder sich im Vergleich zu 2020 nicht verändern konnten. Die hohen Fallzahlen in den jeweiligen Ländern sind natürlich auch auf die hohe Bevölkerung zurückzuschreiben.'''

In [190]:
filtered_df = df[df.year == 2022].groupby(['iso_code','year','location','continent']).sum().reset_index()

fig1_1_2022_cases = px.choropleth(filtered_df, 
            locations = 'iso_code',
            color = 'total_cases', 
            locationmode='ISO-3',
            scope='world',
            height=600,
            color_continuous_scale='purd',
            labels={'iso_code':'Land'
                }
            )
fig1_1_2022_cases.update_traces(hovertext='location')

fig1_1_2022_cases.update_layout(coloraxis_showscale=False,
    title = {'text':"Corona-Infizierte in Summe in 2022", 
        'x':0.5,
        'y':0.95,
        'xanchor': 'center',
        'yanchor': 'top'
        },
    margin={"r":0,"t":0,"l":0,"b":0})

fig1_1_2022_cases.update_geos(fitbounds="locations", visible=False)


In [191]:
fig1_1_2020_cases.write_image("fig1_1_2020_cases.png", width='600', scale=1.5)
fig1_1_2021_cases.write_image("fig1_1_2021_cases.png", width='600', scale=1.5)
fig1_1_2022_cases.write_image("fig1_1_2022_cases.png", width='600', scale=1.5)

#### Abbildung 1.2: Monatlich Tote pro Land im zeitlichen Verlauf(2020,2021,2022)

In [192]:
#text
fig1_1_2020_deaths_title='Wie haben sich die Todeszahlen an Corona in 2020 entwickelt??'
fig1_1_2020_deaths_description='''Die Grafik zeigt, dass die Top 3 Länder auch hier <b> USA, Indien und Brasilien</b> sind. '''

In [193]:
filtered_df = df[df.year == 2020].groupby(['iso_code','year','location','continent']).sum().reset_index()

fig1_1_2020_deaths = px.choropleth(filtered_df, 
            locations = 'iso_code',
            color = 'total_deaths', 
            locationmode='ISO-3',
            scope='world',
            height=600,
            color_continuous_scale='purd',
            labels={'iso_code':'Land'
                }
            )
fig1_1_2020_deaths.update_traces(hovertext='location')

fig1_1_2020_deaths.update_layout(coloraxis_showscale=False,
    title = {'text':"Corona-Tote in Summe in 2020", 
        'x':0.5,
        'y':0.95,
        'xanchor': 'center',
        'yanchor': 'top'
        },
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig1_1_2020_deaths.update_geos(fitbounds="locations", visible=False)


In [194]:
fig1_1_2020_deaths.write_image("fig1_1_2020_deaths.png", width='600', scale=1.5)

In [195]:
filtered_df = df[df.year == 2021].groupby(['iso_code','year','location','continent']).sum().reset_index()

fig1_1_2021_deaths = px.choropleth(filtered_df, 
            locations = 'iso_code',
            color = 'total_deaths', 
            locationmode='ISO-3',
            scope='world',
            height=600,
            color_continuous_scale='purd',
            labels={'iso_code':'Land'
                }
            )
fig1_1_2021_deaths.update_traces(hovertext='location')

fig1_1_2021_deaths.update_layout(coloraxis_showscale=False,
    title = {'text':"Corona-Tote in Summe in 2021", 
        'x':0.5,
        'y':0.95,
        'xanchor': 'center',
        'yanchor': 'top'
        },
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig1_1_2021_deaths.update_geos(fitbounds="locations", visible=False)

In [196]:
#text
fig1_1_2022_deaths_title='Wie haben sich die Todeszahlen bis heute entwickelt??'
fig1_1_2022_deaths_description='''Die Grafik zeigt, dass die Top 3 Länder <b> USA, Indien und Brasilien</b> weiterhin an der Spitze sind. Nichtsdestotrotz ist erkennbar, dass auch in Russland und Mexiko die Todeszahlen stark zugenommen haben im Vergleich zu den Vorjahren.'''

In [197]:
filtered_df = df[df.year == 2022].groupby(['iso_code','year','location','continent']).sum().reset_index()
fig1_1_2022_deaths = px.choropleth(filtered_df, 
            locations = 'iso_code',
            color = 'total_deaths', 
            locationmode='ISO-3',
            scope='world',
            height=600,
            color_continuous_scale='purd',
            labels={'iso_code':'Land'
                }
            )
fig1_1_2022_deaths.update_traces(hovertext='location')

fig1_1_2022_deaths.update_layout(coloraxis_showscale=False,
    title = {'text':"Corona-Tote in Summe in 2022", 
        'x':0.5,
        'y':0.95,
        'xanchor': 'center',
        'yanchor': 'top'
        },
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig1_1_2022_deaths.update_geos(fitbounds="locations", visible=False)

In [198]:
fig1_1_2020_deaths.write_image("fig1_1_2020_deaths.png", width='600', scale=1.5)
fig1_1_2021_deaths.write_image("fig1_1_2021_deaths.png", width='600', scale=1.5)
fig1_1_2022_deaths.write_image("fig1_1_2022_deaths.png", width='600', scale=1.5)

#### Abbildung 2.1: Infizierte in Deutschland in Summe

In [199]:
#text
fig2_1_title='Wie viele Infizierte gibt es in Deutschland in Summe?'
fig2_1_description='''In Deutschland haben sich bereits '''+str(germany["total_cases"].iloc[-1])+''' Menschen mit Corona infiziert.'''

In [200]:
fig2_1 = go.Figure()
fig2_1.add_trace(go.Indicator(mode = "number",value = germany["total_cases"].iloc[-1],title='Infizierte',number = {'valueformat':',',"font":{"size":50,"color":'rgb(229,134,6)'}}))
fig2_1.update_layout(margin=dict(l=0,b=0,r=0,t=0,pad=0), height=200) 

In [201]:
fig2_1.write_image("fig2_1.png", width='600', scale=1.5)

#### Abbildung 2.2: Tote in Deutschland in Summe

In [202]:
#text
fig2_2_title='Wie viele Menschen sind im Zusammenhang mit Corona bereits gestorben?'
fig2_2_description='''In Deutschland sind bereits '''+str(germany["total_deaths"].iloc[-1])+''' Menschen im Zusammenhang mit Corona gestorben. Das entspricht einer Sterberate von über 1%.'''

In [203]:
fig2_2 = go.Figure()
fig2_2.add_trace(go.Indicator(mode = "number",value = germany["total_deaths"].iloc[-1],title='Tote',number = {'valueformat':',',"font":{"size":50,"color":'rgb(237,100,90)'}}))
fig2_2.update_layout(margin=dict(l=0,b=0,r=0,t=0,pad=0), height=200) 

In [204]:
fig2_2.write_image("fig2_2.png", width='600', scale=1.5)

#### Abbildung 3.1: Belegte Betten in Deutschland

In [205]:
#text
fig3_1_title='Wie viele Intensivbetten sind derzeit mit Coronapatienten belegt?'
fig3_1_description='''Über <b> 21.000 Intensivbetten</b> sind derzeit in Deutschland mit Coronapatienten belegt.'''

In [206]:
max_date = max(divi.date)
belegt= divi[(divi.date == max_date)].groupby('date').sum().reset_index()
belegt = belegt.iloc[-1].betten_belegt
fig3_1 = go.Figure()
fig3_1.add_trace(go.Indicator(mode = "number",value =belegt,title="Belegte Betten",number = {'valueformat':',',"font":{"size":50,'color':'rgb(237,100,90)'}}))
fig3_1.update_layout(margin=dict(l=0,b=0,r=0,t=0,pad=0), height=100) 

In [207]:
fig3_1.write_image("fig3_1.png", width='600', scale=1.5)

#### Abbildung 3.2: Freie Betten in Deutschland

In [208]:
#text
fig3_2_title='Wie viele Intensivbetten sind derzeit in Deutschland noch frei?'
fig3_2_description='''Weniger als <b> 4.000 Intensivbetten</b> stehen derzeit in Deutschland zur Verfügung.'''

In [209]:
max_date = max(divi.date)
frei = divi[(divi.date == max_date)].groupby('date').sum().reset_index()
frei = frei.iloc[-1].betten_frei

fig3_2 = go.Figure()
fig3_2.add_trace(go.Indicator(mode = "number",value = frei,title='Freie Betten',number = {'valueformat':',',"font":{"size":50,"color":'rgb(229,134,6)'}}))
fig3_2.update_layout(margin=dict(l=0,b=0,r=0,t=0,pad=0), height=100) 


In [210]:
fig3_2.write_image("fig3_2.png", width='600', scale=1.5)

#### Abbildung 3.3: Anzahl Infizierte in Landkreise ohne freie Intensivbetten


In [211]:
#text
fig3_3_title='Wie vielen Infizierten steht in welchem Landkreisen aktuell kein Bett zur Verfügung?'
fig3_3_description='''Derzeit können in Summe in '''+str(anzahl_landkreise)+''' Landkreisen '''+str(anzahl_infizierte)+''' Infizierte nicht stationär behandelt werden.'''

In [212]:

nichtfrei = divi[(divi.date == max(divi.date))&(divi.betten_frei == 0)&(divi.faelle_covid_aktuell > 0)]
anzahl_landkreise = len(nichtfrei.name)
anzahl_infizierte = sum(nichtfrei.faelle_covid_aktuell)

fig3_3 = px.bar(nichtfrei,y="name",x="faelle_covid_aktuell",orientation="h",color="bundesland_name",text='faelle_covid_aktuell',labels={'bundesland_name':'Bundesland','name':"Landkreis",
                'faelle_covid_aktuell':'Infizierte'},template="plotly_white")
fig3_3.update_traces(textposition='inside')
fig3_3.update_layout(
    title = {'text':'Anzahl Infizierter in Landkreisen ohne freie Intensivbetten',
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    coloraxis_showscale=False,
    barmode='stack',
    xaxis={'title':'','categoryorder':'total descending', 'visible':False,'showticklabels':False},
    yaxis={'title':'','visible': True, 'showticklabels': True },
    showlegend = False

)

In [213]:
fig3_3.write_image("fig3_3.png", width='600', scale=1.5)

#### Abbildung 3.4: Impfstatus

In [214]:
#text
fig3_4_title='Wie viele Menschen haben aktuell schon welche Impfung erhalten??'
fig3_4_description='''Aktuell haben '''+str(sum(status[status.Impfschutz == 'Erstimpfung'].Anzahl))+''' eine Erstimpfung erhalten. '''+str(sum(status[status.Impfschutz == 'Zweitimpfung'].Anzahl))+''' haben eine Zweitimpfung erhalten. '''+str(sum(status[status.Impfschutz == 'Boosterimpfung'].Anzahl))+''' Personen sind schon vollständig geimpft.'''

In [215]:
status = rki.groupby(['Impfschutz','bundesland_name']).sum('Anzahl').reset_index()

fig3_4 = px.bar(status, 
    x='Anzahl', 
    y="bundesland_name", 
    color ="Impfschutz",
    color_discrete_sequence=px.colors.qualitative.Bold, 
    category_orders={"Impfschutz": ["Erstimpfung", "Zweitimpfung", "Boosterimpfung"]},
    labels={'bundesland_name':'Bundesland'
            },
    template="plotly_white"
)
fig3_4.update_layout(
    title = {'text':'Impfstatus',
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    yaxis_title = '',
    coloraxis_showscale=False,
    xaxis={'title': '','categoryorder':'total descending'},
    legend={'title_text':''}
)

In [216]:
fig3_4.write_image("fig3_4.png", width='600', scale=1.5)

#### Abbildung 3.5: Impfquote Erstimpfung

In [217]:
fig3_5_title='Wie hoch ist die Impfquote in den Bundesländern bezogen auf Erstimpfungen??'
fig3_5_description='''Im Durchschnitt haben in Deutschland '''+str(quoten_land.Impfquote_gesamt_min1.mean())+''' Prozent der Bevölkerung eine Erstimpfung erhalten.'''

In [218]:
fig3_5 = px.bar(quoten_land,y="Bundesland",x="Impfquote_gesamt_min1",color="Bundesland",orientation="h",text="Impfquote_gesamt_min1",labels={'bundesland_name':'Bundesland',
                'Impfquote_gesamt_min1':'Erstimpfung'},template="plotly_white")
fig3_5.update_traces(textposition='inside', texttemplate="%{x}%")
fig3_5.update_layout(
    title = {'text':'Impfquote Erstimpfung',
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    xaxis_title = 'Impfquote',
    coloraxis_showscale=False,
    barmode='stack',
    xaxis={'categoryorder':'total descending', 'visible':False,'showticklabels':False},
    yaxis={'title': '','visible': True, 'showticklabels': True },
    showlegend = False
)

In [219]:
fig3_5.write_image("fig3_5.png", width='600', scale=1.5)

#### Abbildung 3.6: Impfquote Boosterimpfung

In [220]:
fig3_6_title='Wie hoch ist die Impfquote in den Bundesländern bezogen auf Boosterimpfungen??'
fig3_6_description='''Im Durchschnitt haben in Deutschland '''+str(quoten_land.Impfquote_gesamt_boost.mean())+''' Prozent der Bevölkerung eine Boosterimpfung erhalten.'''

In [221]:
fig3_6 = px.bar(quoten_land,y="Bundesland",x="Impfquote_gesamt_boost",color="Bundesland",orientation="h", text='Impfquote_gesamt_boost',labels={'bundesland_name':'Bundesland',
                'Impfquote_gesamt_boost':'Geboostert'},template="plotly_white")
fig3_6.update_traces(textposition='inside',texttemplate="%{x}%")
fig3_6.update_layout(
    title = {'text':'Impfquote Boosterimpfung',
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    xaxis_title = 'Impfquote',
    coloraxis_showscale=False,
    barmode='stack',
    xaxis={'categoryorder':'total descending', 'visible':False,'showticklabels':False},
    yaxis={'title': '','visible': True, 'showticklabels': True },
    showlegend = False
)

In [222]:
fig3_6.write_image("fig3_6.png", width='600', scale=1.5)

#### Abbildung 3.7: Forecast

In [223]:
fig3_7_title='Wie entwickeln sich die Fallzahlen in Deutschland?'
fig3_7_description=''''''

In [224]:

cases_prophet = divi.copy()
cases_prophet = cases_prophet[['date','faelle_covid_aktuell']].groupby('date').sum().reset_index()
cases_prophet.columns = ['ds','y']

m = Prophet(interval_width=0.95,changepoint_prior_scale=0.01,seasonality_mode='multiplicative',daily_seasonality=False,weekly_seasonality=True,yearly_seasonality=True,changepoint_range=0.6)
m.fit(cases_prophet)
future = m.make_future_dataframe(periods=360)
forecast = m.predict(future)
forecast['y'] = cases_prophet['y']

pred = forecast[forecast.y.isna()]
pred.loc[pred['yhat'] < 0, 'yhat'] = 0
pred.loc[pred['yhat_lower'] < 0, 'yhat_lower'] = 0
pred.loc[pred['yhat_upper'] < 0, 'yhat_upper'] = 0

fig3_7 = go.Figure()

# Add traces

fig3_7.add_trace(go.Scatter(
        name='Infizierte',
        mode = 'lines',
        x = list(pred['ds']),
        y = list(pred['yhat']),
        marker=dict(
            #  color='red',
            line=dict(width=3)
        )
    ))

fig3_7.add_trace(go.Scatter(
        name= 'Unteres Konfidenzintervall',
        mode = 'lines',
        x = list(pred['ds']),
        y = list(pred['yhat_lower']),
        line= dict(color='#57b88f')
    ))

fig3_7.add_trace(go.Scatter(
    name = 'Oberes Konfidenzintervall',
    mode = 'lines',
    x = list(pred['ds']),
    y = list(pred['yhat_upper']),
    line= dict(color='#57b88f'),
    fill = 'tonexty'
))

fig3_7.update_layout(

    xaxis_title='',
    yaxis_title='',
    template='plotly_white'

)

Initial log joint probability = -32.5285
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1491.41     0.0018985       292.713           1           1      136   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       1501.92   1.16556e-06        267.61     0.03251      0.4053      275   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     235       1502.12   0.000171363       370.371   4.535e-07       0.001      364  LS failed, Hessian reset 
     297       1502.17    6.4861e-06       340.212   1.929e-08       0.001      496  LS failed, Hessian reset 
     299       1502.18   4.67424e-06       257.985           1           1      498   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     389       1502.41   0.000162273       316.601   4.889e-07       0.001      672  LS failed, Hessian rese

/Users/hannaziegler/opt/anaconda3/envs/jbook/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [225]:
fig3_7.write_image("fig3_7.png", width='600', scale=1.5)

#### Render Report

In [243]:
# Template handling
env = jinja2.Environment(loader=jinja2.FileSystemLoader(searchpath=''))
template = env.get_template('report_template.html')

In [244]:
output = template.render(
                        fig1_1_2020_cases_title=fig1_1_2020_cases_title, fig1_1_2020_cases_description=fig1_1_2020_cases_description,
                        fig1_1_2022_cases_title=fig1_1_2022_cases_title, fig1_1_2022_cases_description=fig1_1_2022_cases_description,
                        fig1_1_2020_deaths_title=fig1_1_2020_deaths_title, fig1_1_2020_deaths_description=fig1_1_2020_deaths_description,
                        fig1_1_2022_deaths_title=fig1_1_2022_deaths_title, fig1_1_2022_deaths_description=fig1_1_2022_deaths_description,
                        fig2_1_title=fig2_1_title, fig2_1_description=fig2_1_description,
                        fig2_2_title=fig2_2_title, fig2_2_description=fig2_2_description,
                        fig3_1_title=fig3_1_title, fig3_1_description=fig3_1_description,
                        fig3_2_title=fig3_2_title, fig3_2_description=fig3_2_description,
                        fig3_3_title=fig3_3_title, fig3_3_description=fig3_3_description,
                        fig3_4_title=fig3_4_title, fig3_4_description=fig3_4_description,
                        fig3_5_title=fig3_5_title, fig3_5_description=fig3_5_description,
                        fig3_6_title=fig3_6_title, fig3_6_description=fig3_6_description,
                        fig3_7_title=fig3_7_title, fig3_7_description=fig3_7_description)
                        

In [245]:
# Write the HTML file
with open('report.html', 'w') as f:
    f.write(output)

In [246]:
#convert html to pdf
with open('report.pdf', "w+b") as result_file:
    pisa.CreatePDF(
        src=output,  # HTML to convert
        dest=result_file)  # File handle to receive result